In [17]:
import os
import scipy
import numpy as np
import pandas as pd
from pathlib import Path
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from sklearn import set_config
set_config(display = 'diagram')
from pandas.api.types import infer_dtype


# Scikit Learn import
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.pipeline import FeatureUnion
# from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn import cluster
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputRegressor


# Category Encoder
import category_encoders as ce
# pyod
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.cof import COF
from pyod.models.iforest import IForest
from pyod.models.hbos import HBOS
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA as PCA_pyod
from pyod.models.mcd import MCD
from pyod.models.sod import SOD
from pyod.models.sos import SOS
# model
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from imblearn.over_sampling import SMOTE

import re

import vnquant.DataLoader as dl
from datetime import datetime, timedelta
import pytz
from pathlib import Path


In [18]:
init_date = '2020-01-01'
start_date = '2021-01-01'
end_date = '2022-03-20'
symbols = ["HSG", "HPG", "NKG"]
def get_data(symbol, start_date = start_date, end_date = end_date):
    loader = dl.DataLoader(symbols=[symbol], start=start_date, end=end_date, minimal=True)
    data = loader.download()
    data.columns = [col[0] for col in data.columns]
    data = data.reset_index()
    data['symbol'] = symbol
    data['target_day_1'] = data.close.shift(-1)
    data['target_day_2'] = data.close.shift(-2)
    data['target_day_3'] = data.close.shift(-3)
    data['target_day_4'] = data.close.shift(-4)
    data['target_day_5'] = data.close.shift(-5)
    return data
df = pd.concat([get_data(symbol) for symbol in symbols], axis = 0)
df = df.sort_values(by=['date'])
df.tail(10)

/home/weihan/development/anaconda3/lib/python3.9/site-packages/vnquant-0.0.3-py3.9.egg/vnquant/DataLoader.py:254: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  stock_data['change_perc1'], stock_data['change_perc2'] = stock_data['change_perc'].apply(utils.split_change_col).str
2022-03-21 00:50:55,892 : INFO : data HSG from 2021-01-01 to 2022-03-20 have already cloned!
2022-03-21 00:50:59,244 : INFO : data HPG from 2021-01-01 to 2022-03-20 have already cloned!
2022-03-21 00:51:03,041 : INFO : data NKG from 2021-01-01 to 2022-03-20 have already cloned!


,date,high,low,open,close,adjust,volume,symbol,target_day_1,target_day_2,target_day_3,target_day_4,target_day_5
294,2022-03-15,38.70,36.70,38.00,37.20,37.20,12933900,HSG,37.35,36.65,37.85,NaN,NaN
296,2022-03-16,47.15,46.20,46.90,46.20,46.20,11438800,HPG,45.95,46.50,NaN,NaN,NaN
295,2022-03-16,47.45,46.45,47.25,46.60,46.60,5215600,NKG,45.25,46.20,NaN,NaN,NaN
295,2022-03-16,38.05,37.15,37.60,37.35,37.35,6862000,HSG,36.65,37.85,NaN,NaN,NaN
296,2022-03-17,47.20,44.50,46.90,45.25,45.25,12281200,NKG,46.20,NaN,NaN,NaN,NaN
296,2022-03-17,37.90,36.55,37.70,36.65,36.65,10006100,HSG,37.85,NaN,NaN,NaN,NaN
297,2022-03-17,46.70,45.90,46.60,45.95,45.95,15172700,HPG,46.50,NaN,NaN,NaN,NaN
298,2022-03-18,46.85,46.05,46.20,46.50,46.50,18336600,HPG,NaN,NaN,NaN,NaN,NaN
297,2022-03-18,38.10,36.65,37.10,37.85,37.85,13268600,HSG,NaN,NaN,NaN,NaN,NaN
297,2022-03-18,47.00,44.60,45.45,46.20,46.20,7909200,NKG,NaN,NaN,NaN,NaN,NaN


In [19]:
numeric_columns = ["high", "low", "open", "close", "volume"]
category_columns = ['symbol']
time_columns = ['date']
feature_columns = numeric_columns+category_columns+time_columns
target_columns = ['target_day_1', 'target_day_2', 'target_day_3', 'target_day_4', 'target_day_5']
# X = df.loc[df[target_columns].isnull().any(axis = 1)].drop(columns=target_columns)
# y = df.loc[df[target_columns].notnull().any(axis = 1)][target_columns]
# test = df.loc[df[target_columns].isnull().any(axis = 1)]

In [20]:
class TimePreprocess(BaseEstimator, TransformerMixin):
    def __init__(self, list_of_features=["day", "dayofweek", "month"],):
        super().__init__()
        self.time_features = []
        self.list_of_features = list_of_features
    def fit(self, X, y = None):
        for col in X.columns:
            try:
                X[col] = pd.to_datetime(X[col])
                self.time_features.append(col)
            except:
                logger.error('Time columns cannot be convert to datetime')
        return self
    def transform(self, X):
        columns = []
        for col in self.time_features:
            X[col] = pd.to_datetime(X[col])
            if 'day' in self.list_of_features:
                columns.append(col+'_day')
                X[columns[-1]] = X[col].dt.day
            if 'month' in self.list_of_features:
                columns.append(col+'_month')
                X[columns[-1]] = X[col].dt.month
            if 'dayofweek' in self.list_of_features:
                columns.append(col+'_dayofweek')
                X[columns[-1]] = X[col].dt.dayofweek
        return X[columns]
    def fit_transform(self, X, y = None):
        self.fit(X, y)
        return self.transform(X)


numeric_preprocess  = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler_quantile', QuantileTransformer()),
    # ('scaler', MinMaxScaler())
])
category_preprocess = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', ce.OneHotEncoder(use_cat_names = True)),
])

In [27]:


preprocessing = ColumnTransformer(transformers=[
        ('numeric_preprocess', numeric_preprocess, numeric_columns),
        ('category_columns', category_preprocess, category_columns),
        ('time_preprocess', TimePreprocess(), time_columns),
        # ('hold_feature', HoldFeature(), hold_columns),
    ],
                                      n_jobs = -1
                                     )

pipe = Pipeline(steps = [
    ('preprocessing', preprocessing),
    ('estimator', CatBoostRegressor())
])
# pipe.fit(X, y)
params = {
    'estimator': [CatBoostRegressor(verbose = 0),
                  # LGBMRegressor(verbose = 0),
                  # XGBRegressor()
                 ]
}
grid =  GridSearchCV(estimator = pipe, 
                     param_grid = params,
                     scoring = 'neg_mean_squared_error',
                     cv = TimeSeriesSplit(n_splits=5),
                     n_jobs = -1
                    )
for target_col in target_columns:
    X = df.loc[df[target_col].notnull(), feature_columns]
    y = df.loc[df[target_col].notnull(), target_col]
    grid.fit(X, y)
    print(grid.cv_results_['mean_test_score'])
    X_test = df.loc[df[target_col].isnull(), feature_columns]
    df.loc[df[target_col].isnull(), target_col] = grid.best_estimator_.predict(X_test)

/home/weihan/development/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (299). n_quantiles is set to n_samples.
  warnings.warn(
/home/weihan/development/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (448). n_quantiles is set to n_samples.
  warnings.warn(
/home/weihan/development/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (597). n_quantiles is set to n_samples.
  warnings.warn(
/home/weihan/development/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (150). n_quantiles is set to n_samples.
  warnings.warn(
/home/weihan/development/anaconda3/lib/python3.9/site-packages/sklea

[-16.11796104]


ValueError: Found array with 0 sample(s) (shape=(0, 5)) while a minimum of 1 is required.

In [26]:
df.tail()

,date,high,low,open,close,adjust,volume,symbol,target_day_1,target_day_2,target_day_3,target_day_4,target_day_5
296,2022-03-17,37.90,36.55,37.70,36.65,36.65,10006100,HSG,37.850000,NaN,NaN,NaN,NaN
297,2022-03-17,46.70,45.90,46.60,45.95,45.95,15172700,HPG,46.500000,NaN,NaN,NaN,NaN
298,2022-03-18,46.85,46.05,46.20,46.50,46.50,18336600,HPG,46.436290,NaN,NaN,NaN,NaN
297,2022-03-18,38.10,36.65,37.10,37.85,37.85,13268600,HSG,37.549697,NaN,NaN,NaN,NaN
297,2022-03-18,47.00,44.60,45.45,46.20,46.20,7909200,NKG,45.990629,NaN,NaN,NaN,NaN


In [ ]:
grid.cv_results_

In [ ]:
X_test['target'] = grid.best_estimator_.predict(X_test)

In [ ]:
X_test

In [51]:
import psycopg

conn = psycopg.connect(dbname = "postgres", user = "user", password = "pass", host = "127.0.0.1", port = "5433")
cur = conn.cursor()

In [41]:
def query(query_string, value_dict=None):
    try:
        if value_dict != None:
            cur.execute(query_string, value_dict)
        else: 
            cur.execute(query_string)
        conn.commit()
        return cur
    except Exception as e:
        print('Error. Rollback connection')
        conn.rollback()
        print(e)
    return cur

In [42]:
create_query = """
CREATE TABLE IF NOT EXISTS D_USER_NAME (
    USER_ID VARCHAR NOT NULL,
    NAME VARCHAR NOT NULL
);
"""
query(create_query)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=0.0.0.0 port=5433 user=user database=postgres) at 0x7f9fb1e50f60>

In [43]:
cur.execute("INSERT INTO D_USER_NAME (user_id, name) VALUES (%s, %s)", ("12345", "tonne"))

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=0.0.0.0 port=5433 user=user database=postgres) at 0x7f9fb1e50f60>

In [44]:
cur.execute("INSERT INTO D_USER_NAME (user_id, name) VALUES (%s, %s)", ("12346", "tonnene"))

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=0.0.0.0 port=5433 user=user database=postgres) at 0x7f9fb1e50f60>